In [792]:
import gurobipy as gp
from gurobipy import GRB
from operator import itemgetter
import itertools
import math
import numpy as np 
import json
from prettytable import PrettyTable


class MultiDimensionalArrayEncoder(json.JSONEncoder):
    def encode(self, obj):
        def hint_tuples(item):
            if isinstance(item, tuple):
                return {'__tuple__': True, 'items': item}
            if isinstance(item, list):
                return [hint_tuples(e) for e in item]
            if isinstance(item, dict):
                return {key: hint_tuples(value) for key, value in item.items()}
            else:
                return item

        return super(MultiDimensionalArrayEncoder, self).encode(hint_tuples(obj))
def hinted_tuple_hook(obj):
    if '__tuple__' in obj:
        return tuple(obj['items'])
    else:
        return obj
enc = MultiDimensionalArrayEncoder()

In [793]:
def formar_asignaciones(profesores_por_curso: dict):
    lista_asignaciones = []
    for curso in profesores_por_curso:
        for tupla in profesores_por_curso[curso]:
            lista_asignaciones.append((curso, tupla[0], tupla[1]))
    return lista_asignaciones

def formar_asignaciones_multiples(clase_conjunta):
    lista_asignaciones_multiples = []
    cantidad_cursos_simultaneos = {}
    for cs,p,k in clase_conjunta:
        for c in cs:
            lista_asignaciones_multiples.append((c,p,k))
            cantidad_cursos_simultaneos[c,p,k] = len(cs)
    return lista_asignaciones_multiples, cantidad_cursos_simultaneos

In [794]:
model = gp.Model("Generación de horarios de colegio")
# model.setParam('OutputFlag', 1)
f = open('parametros2.json',encoding='utf8')

M = 10000
data = json.load(f, object_hook=hinted_tuple_hook)
dias = data['dias']
modulos = data['modulos']
ramos_con_modulos_seguidos = data['ramos_con_modulos_seguidos']
combinaciones = data['combinaciones']
horarios = data['horarios']
profesores_por_curso = data['profesores_por_curso']
ramos_por_curso = data['ramos_por_curso']
carga_profesores = data['carga_profesores']
multiples_cursos = data['multiples_cursos']
# mutiples_profesores = data['multiples_profesores']
# multiples_cursos_y_profesores = data['multiples_cursos_y_profesores']

# # # ! DATOS HECHIZOS
# cantidad_profesores_ramo = {
#     "Lenguaje": 10,
#     "Matemáticas": 10,
#     "Historia": 7,
#     "Artes_Visuales": 5,
#     "Música": 5,
#     "Educación_Física": 8,
#     "Orientación": 4,
#     "Tecnología": 4,
#     "Religión": 5,
#     "Ciencias_Naturales": 7,
#     "Inglés": 7,
#     "Consejo_de_Curso": 1,
#     "Filosofía": 1
# }
# cantidad_cursos_profesor = {
#     "Lenguaje": 4,
#     "Matemáticas": 4,
#     "Historia": 4,
#     "Artes_Visuales": 6,
#     "Música": 6,
#     "Educación_Física": 4,
#     "Orientación": 10,
#     "Tecnología": 10,
#     "Religión": 7,
#     "Ciencias_Naturales": 4,
#     "Inglés": 4,
#     "Consejo_de_Curso": 10,
#     "Filosofía": 10
# }

# import simulador
# ramos_por_curso = simulador.generar_horas_libres(ramos_por_curso)
# lista_profesores_para_asignaciones, profesores = simulador.generar_profesores(7, 74, cantidad_profesores_ramo, cantidad_cursos_profesor)
# profesores_por_curso = simulador.generar_asignaciones(lista_profesores_para_asignaciones, ramos_por_curso)

asignaciones_normales = formar_asignaciones(profesores_por_curso)
asignaciones_multiples, cantidad_de_cursos_simultaneos = formar_asignaciones_multiples(multiples_cursos)


asignaciones = asignaciones_normales

x = model.addVars(asignaciones,dias,modulos,vtype=GRB.BINARY, name="x")
s = model.addVars(asignaciones,dias,vtype=GRB.BINARY, name="s")

In [795]:
model.addConstrs((sum(x[c,p,k,d,j] for p,k in profesores_por_curso[c]) == 1 \
    for c in horarios for d in horarios[c] for j in horarios[c][d]),name="R1");

In [796]:
model.addConstrs((sum(x[c,p,k,d,j] for d in dias for j in modulos) == ramos_por_curso[c][k] \
    for c in profesores_por_curso for p,k in profesores_por_curso[c]),name="R2");

In [797]:
model.addConstrs((sum(x[c,p,k,d,j] for p,k in profesores_por_curso[c]) == 0 \
    for c in horarios for d in horarios[c] for j in list(set(modulos) - set(horarios[c][d]))),name="R3");

In [798]:
model.addConstrs((sum(x[c,p,k,d,j] for c,k in carga_profesores[p][q] if c != c2) + \
     x[c2,p,k2,d,j] <= int(q) \
     for cs,p2,k2 in multiples_cursos for c2 in [cs[a] for a in range(len(cs)) for b in range(len(cs)) if a < b]
     for p in list(carga_profesores.keys()) for q in carga_profesores[p] for d in dias for j in modulos if p == p2),name="R4a");

In [799]:
model.addConstrs((sum(x[c,p,k,d,j] for c,k in carga_profesores[p][q]) <= int(q) \
     for p in list(carga_profesores.keys()) for q in carga_profesores[p] for d in dias for j in modulos),name="R4b");

In [800]:
model.addConstrs((x[c1,p,k,d,j] - x[c2,p,k,d,j] == 0 \
    for cs,p,k in multiples_cursos for c1,c2 in [(cs[a],cs[b]) for a in range(len(cs)) for b in range(len(cs)) if a < b]
    for d in dias for j in modulos),name="R5");

In [801]:
model.addConstrs((sum(x[c,p,k,d,j] for j in modulos) <= 2 for d in dias for c in horarios for p,k in profesores_por_curso[c]),name="R6");

In [802]:
model.addConstrs((x[c,p,k,d,j] + x[c,p,k,d,z] <= 1 for c,p,k in asignaciones for d in dias \
    for j,z in list(set([(a+1,b+1) for a in horarios[c][d] for b in horarios[c][d] if a < b]) - set(combinaciones))),name="R7");

In [803]:
model.addConstrs((M * s[c,p,k,d] >= sum(x[c,p,k,d,j] for j in modulos) for c,p,k in asignaciones for d in dias),name="RS1");

In [804]:
model.addConstrs((sum(s[c,p,k,d] for d in dias) <= math.ceil(ramos_por_curso[c][k] / 2) \
    for c in horarios for p,k in profesores_por_curso[c] if k in ramos_con_modulos_seguidos),name="RS2");

In [805]:
model.setObjective(0, GRB.MINIMIZE)

model.write('model.lp')
model.update()

# model.computeIIS()
# removed =[]
# for c in model.getConstrs():
#     if c.IISConstr:
#         print('%s' % c.constrName)
#         # Remove a single constraint from the model
#         removed.append(str(c.constrName))
#         model.remove(c)

model.optimize()
model.write('out.sol')

Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 5488 rows, 2000 columns and 18330 nonzeros
Model fingerprint: 0x81a1f223
Variable types: 0 continuous, 2000 integer (2000 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+04]
  Objective range  [0e+00, 0e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 8e+00]
Presolve removed 3988 rows and 478 columns
Presolve time: 0.03s
Presolved: 1500 rows, 1522 columns, 8845 nonzeros
Variable types: 0 continuous, 1522 integer (1522 binary)

Root relaxation: objective 0.000000e+00, 812 iterations, 0.02 seconds (0.01 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

H    0     0                       0.0000000    0.00000  0.00%     -    0s
     0     0    0.00000    0  166    0.00000    0.00000  0.00%     -  

In [807]:
def leer_output():
    lista_final = []
    with open('out.sol', 'r') as file:
        lines = file.readlines()
        for line in lines:
            if line[-2] == '1':
                line = line.strip('\n')
                lista_final.append(line)
    # print(lista_final)
    diccionario_dias = {
        'Lunes': 1,
        'Martes': 2,
        'Miércoles': 3,
        'Jueves': 4,
        'Viernes': 5
    }

    vuelta = {
        1: 'Lunes',
        2: 'Martes',
        3: 'Miércoles',
        4: 'Jueves',
        5: 'Viernes'
    }

    profesores_final = []
    for elem in lista_final:
        if elem[0] == "x":
            elem = elem[2:-3]
            elem = elem.split(',')
            profesores_final.append(elem)
    for i in range(len(profesores_final)):
        profesores_final[i][3] = diccionario_dias[profesores_final[i][3]]
    profesores_final = sorted(profesores_final, key= lambda x: x[4])
    profesores_final = sorted(profesores_final, key= lambda x: x[3])
    profesores_final = sorted(profesores_final, key= lambda x: x[0])

    for i in range(len(profesores_final)):
        profesores_final[i][3] = vuelta[profesores_final[i][3]]
    return profesores_final

def generar_horario(curso):

    clases_curso = []
    dias_semana = []
    ultimo_dia = None
    horario = PrettyTable(dias_semana)
    lista_auxiliar = []
    lista_dia = None
    for elem in leer_output():    
        if elem[0] == curso:
            clases_curso.append([elem[2], elem[3], elem[4], elem[1]])
            if elem[3] not in dias_semana:
                dias_semana.append(elem[3])
            if elem[3] != ultimo_dia:
                lista_auxiliar.append(lista_dia)
                lista_dia = []
                ultimo_dia = elem[3]
                lista_dia.append(ultimo_dia)
                elem[2] = elem[2].replace('_', ' ')
                elem[1] = elem[1].replace('_', ' ')
                lista_dia.append(f'{elem[2]} \n {elem[1]}')
            else:
                elem[2] = elem[2].replace('_', ' ')
                elem[1] = elem[1].replace('_', ' ')
                lista_dia.append(f'{elem[2]} \n {elem[1]}')

    lista_auxiliar.append(lista_dia)
    lista_auxiliar = lista_auxiliar[1:]
    len_maxima = 0
    for dia in lista_auxiliar:
        if len(dia) > len_maxima:
            len_maxima = len(dia)

    horario.add_column('', [i+1 for i in range(len_maxima - 1)])

    for i in range(len(lista_auxiliar)):
        for j in range(len_maxima - len(lista_auxiliar[i])):
            lista_auxiliar[i].append('')
        horario.add_column(lista_auxiliar[i][0], lista_auxiliar[i][1:], align='c')
    horario.format = True   
    print(f'Horario del curso {curso}: ')   
    return horario

curso = '2B'
generar_horario(curso)

Horario del curso 2B: 


,Lunes,Martes,Miércoles,Jueves,Viernes
1,Matemáticas Maradona,Educación Física Maradona,Religión Perking,Lenguaje Perking,Historia Maradona
2,Artes Visuales Maradona,Educación Física Maradona,Artes Visuales Maradona,Matemáticas Maradona,Tecnología Mbappé
3,Ciencias Naturales Perking,Música Perking,Artes Visuales Maradona,Matemáticas Maradona,Música Perking
4,Educación Física Maradona,Lenguaje Perking,Matemáticas Maradona,Música Perking,Matemáticas Maradona
5,Educación Física Maradona,Matemáticas Maradona,Orientación Mbappé,Historia Maradona,Matemáticas Maradona
6,Lenguaje Perking,Historia Maradona,Lenguaje Perking,Lenguaje Perking,Lenguaje Perking
7,Lenguaje Perking,Orientación Mbappé,Ciencias Naturales Perking,Ciencias Naturales Perking,Lenguaje Perking
8,,Ciencias Naturales Perking,Religión Perking,Tecnología Mbappé,
